In [53]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
import cv2
import tensorflow as tf
from tensorflow import keras

In [54]:
DATASET_PATH = '/content/drive/My Drive/University Of Stirling/Dissertation/retinal-rec/Datasets/APTOS2019'
TRAIN_PATH = DATASET_PATH + "/train_images/"
TRAIN_PREP_PATH = DATASET_PATH + "/train_preprocessed/"
classes = [ "No DR", "Mild", "Moderate", "Severe", "Proliferative DR" ]
N_CLASSES = 5
IMG_SIZE = (512, 512)

In [55]:
from google.colab import drive
import os

drive.mount('/content/drive')

os.chdir(DATASET_PATH)
print("CWD:",os.getcwd())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
CWD: /content/drive/My Drive/University Of Stirling/Dissertation/retinal-rec/Datasets/APTOS2019


In preparation for the NN, it is necessary to create the correct directory structure.

In [56]:
if not os.path.exists(DATASET_PATH + "/train"):
  os.mkdir(DATASET_PATH + "/train")
  os.mkdir(DATASET_PATH + "/train/0")
  os.mkdir(DATASET_PATH + "/train/1")
  os.mkdir(DATASET_PATH + "/train/2")
  os.mkdir(DATASET_PATH + "/train/3")
  os.mkdir(DATASET_PATH + "/train/4")

if not os.path.exists(DATASET_PATH + "/validation"):
  os.mkdir(DATASET_PATH + "/validation")
  os.mkdir(DATASET_PATH + "/validation/0")
  os.mkdir(DATASET_PATH + "/validation/1")
  os.mkdir(DATASET_PATH + "/validation/2")
  os.mkdir(DATASET_PATH + "/validation/3")
  os.mkdir(DATASET_PATH + "/validation/4")

if not os.path.exists(DATASET_PATH + "/test"):
  os.mkdir(DATASET_PATH + "/test")
  os.mkdir(DATASET_PATH + "/test/0")
  os.mkdir(DATASET_PATH + "/test/1")
  os.mkdir(DATASET_PATH + "/test/2")
  os.mkdir(DATASET_PATH + "/test/3")
  os.mkdir(DATASET_PATH + "/test/4")


In [57]:
from sklearn.model_selection import train_test_split
import shutil

def read_dataset():
  df = pd.read_csv(DATASET_PATH + "/train.csv")
  
  # Train-test split
  train, test = train_test_split(df, test_size=.2)
  train, valid = train_test_split(train, test_size=.3)

  for image in train.values:
    shutil.copyfile(TRAIN_PREP_PATH + image[0] + ".png", DATASET_PATH + "/train/" + str(image[1]) + "/" + image[0] + ".png")

  for image in valid.values:
    shutil.copyfile(TRAIN_PREP_PATH + image[0] + ".png", DATASET_PATH + "/validation/" + str(image[1]) + "/" + image[0] + ".png")

  for image in test.values:
    shutil.copyfile(TRAIN_PREP_PATH + image[0] + ".png", DATASET_PATH + "/test/" + str(image[1]) + "/" + image[0] + ".png")
  
  return (train, valid, test)


train, valid, test = read_dataset()

print(train.shape, valid.shape, test.shape)


(2050, 2) (879, 2) (733, 2)


In [58]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

df = pd.read_csv(DATASET_PATH + "/train.csv")
  
# Train-test split
#train, test = train_test_split(df, test_size=.2)

# create a data generator
datagen = ImageDataGenerator()

# load and iterate training dataset
train_it = datagen.flow_from_directory(DATASET_PATH + "/train/", class_mode='categorical', batch_size=32)
valid_it = datagen.flow_from_directory(DATASET_PATH + "/validation/", class_mode='categorical', batch_size=32)

model=keras.models.Sequential([
    keras.layers.Conv2D(filters=128, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(256,256,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3)),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(1024,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1024,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(5,activation='softmax')  
])

#model.build((256,512,512,3))
model.summary()

# Compile the model
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=tf.keras.optimizers.Adam(learning_rate = 0.001), metrics=["accuracy"], run_eagerly=True)


model.fit(train_it, validation_data=valid_it, epochs=10, verbose=1) 

Found 2050 images belonging to 5 classes.
Found 879 images belonging to 5 classes.
Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_61 (Conv2D)          (None, 62, 62, 128)       46592     
                                                                 
 batch_normalization_60 (Bat  (None, 62, 62, 128)      512       
 chNormalization)                                                
                                                                 
 max_pooling2d_37 (MaxPoolin  (None, 31, 31, 128)      0         
 g2D)                                                            
                                                                 
 conv2d_62 (Conv2D)          (None, 31, 31, 256)       819456    
                                                                 
 batch_normalization_61 (Bat  (None, 31, 31, 256)      1024      
 chNormalization)                   

In [60]:
print("class 2")
num_samples = 20
for file in os.listdir(DATASET_PATH + "/test/2"):
    img = cv2.imread(DATASET_PATH + "/test/2/" + file)
    img = cv2.resize(img, (256,256), interpolation = cv2.INTER_AREA)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = np.expand_dims(img, axis=0)
    prediction = model.predict(x=[img])
    print(np.argmax(prediction))
    num_samples = num_samples-1
    if(num_samples == 0): 
      break

class 2
0
0
0
0
0
0
0
0
0
0
2
0
0
0
4
0
0
0
0
2
